In [ ]:
import os
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

# Caminhos
BASE_PATH = r"C:\Users\luana\Desktop\Squad04-Semantic-Segmentation-Drone-Dataset\dataset\binary_dataset\binary_dataset"
MASK_DIR = os.path.join(BASE_PATH, "images_semantic")

assert os.path.exists(MASK_DIR), f"Diretório não encontrado: {MASK_DIR}"

# Mapeamento para classes binárias
binary_classes = {
    0: {0, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20},  # Background
    1: {1, 2, 3, 4, 9}  # Objeto
}

def map_to_binary(value):
    for binary_val, original_vals in binary_classes.items():
        if value in original_vals:
            return binary_val
    return 0  # Padrão: background

records = []

# Processamento das máscaras
for mask_name in tqdm(os.listdir(MASK_DIR), desc="Processando máscaras"):
    if not mask_name.lower().endswith('.png'):
        continue
    
    mask_path = os.path.join(MASK_DIR, mask_name)
    
    try:
        with Image.open(mask_path) as mask:
            width, height = mask.size
            mask_array = np.array(mask)
            binary_mask = np.vectorize(map_to_binary)(mask_array)

            total_pixels = binary_mask.size
            pixels_background = np.sum(binary_mask == 0)
            pixels_object = np.sum(binary_mask == 1)

            tem_background = pixels_background > 0
            tem_objeto = pixels_object > 0

            proporcao_objeto = round(pixels_object / total_pixels, 4)

            # Define faixa da proporção
            if proporcao_objeto == 0:
                faixa = "0%"
            elif proporcao_objeto < 0.01:
                faixa = "<1%"
            elif proporcao_objeto < 0.1:
                faixa = "1-10%"
            elif proporcao_objeto < 0.25:
                faixa = "10-25%"
            elif proporcao_objeto < 0.5:
                faixa = "25-50%"
            elif proporcao_objeto < 0.75:
                faixa = "50-75%"
            else:
                faixa = ">75%"

            records.append({
                "arquivo": mask_name,
                "largura": width,
                "altura": height,
                "total_pixels": total_pixels,
                "pixels_background": pixels_background,
                "pixels_objeto": pixels_object,
                "proporcao_background": round(pixels_background / total_pixels, 4),
                "proporcao_objeto": proporcao_objeto,
                "somente_background": tem_background and not tem_objeto,
                "somente_objeto": tem_objeto and not tem_background,
                "background_e_objeto": tem_background and tem_objeto,
                "faixa_proporcao_objeto": faixa
            })

    except Exception as e:
        print(f"Erro ao processar {mask_name}: {str(e)}")

# Criação do DataFrame
df = pd.DataFrame(records)

# Salvar CSV completo
csv_path = os.path.join(BASE_PATH, "metadata_binario.csv")
df.to_csv(csv_path, index=False, encoding='utf-8-sig')

# Resumo
print("\n Resumo estatístico:")
print(f"Total de imagens: {len(df)}")
print(f"Imagens com objeto: {df['background_e_objeto'].sum()} ({df['background_e_objeto'].mean():.1%})")
print(f"Proporção média de objeto: {df['proporcao_objeto'].mean():.4f}")

print("\n Resumo por tipo de imagem:")
print(df[["somente_background", "somente_objeto", "background_e_objeto"]].sum())

print("\n Faixas de proporção de objeto:")
print(df["faixa_proporcao_objeto"].value_counts().sort_index())

print(f"\n Arquivo CSV salvo em: {csv_path}")

# Verifica resoluções únicas
print("\n Resoluções únicas encontradas:")
print(df[['largura', 'altura']].drop_duplicates())

# Verifica se há resoluções inconsistentes
resolucao_esperada = (512, 512)
df_dim_errada = df[(df['largura'] != resolucao_esperada[0]) | (df['altura'] != resolucao_esperada[1])]
if not df_dim_errada.empty:
    print(f"\n Imagens com dimensões diferentes de {resolucao_esperada}:")
    print(df_dim_errada[['arquivo', 'largura', 'altura']])


Processando máscaras: 100%|██████████| 400/400 [01:45<00:00,  3.79it/s]


📊 Resumo estatístico:
Total de imagens: 400
Imagens com objeto: 400 (100.0%)
Proporção média de objeto: 0.7562

📦 Resumo por tipo de imagem:
somente_background       0
somente_objeto           0
background_e_objeto    400
dtype: int64

📁 Faixas de proporção de objeto:
faixa_proporcao_objeto
25-50%     20
50-75%    163
>75%      217
Name: count, dtype: int64

✅ Arquivo CSV salvo em: C:\Users\luana\Desktop\Squad04-Semantic-Segmentation-Drone-Dataset\dataset\binary_dataset\binary_dataset\metadata_binario.csv

📐 Resoluções únicas encontradas:
   largura  altura
0      960     736

⚠️ Imagens com dimensões diferentes de (512, 512):
     arquivo  largura  altura
0    000.png      960     736
1    001.png      960     736
2    002.png      960     736
3    003.png      960     736
4    004.png      960     736
..       ...      ...     ...
395  592.png      960     736
396  593.png      960     736
397  594.png      960     736
398  596.png      960     736
399  598.png      960     736

[40

Cada linha representa uma imagem de máscara segmentada, com as seguintes colunas no arquivo principal (metadata_binario.csv ou .xlsx):

### 🗂️ Metadados por imagem

- **`arquivo`**: Nome do arquivo da máscara (.png).
- **`largura`**: Largura da imagem em pixels.
- **`altura`**: Altura da imagem em pixels.
- **`total_pixels`**: Total de pixels da imagem (largura × altura).
- **`pixels_background`**: Número de pixels da classe 0 (background).
- **`pixels_objeto`**: Número de pixels da classe 1 (objeto de interesse).
- **`proporcao_background`**: Proporção de pixels de background sobre o total.
- **`proporcao_objeto`**: Proporção de pixels de objeto sobre o total.
- **`somente_background`**: `True` se a imagem contém apenas background.
- **`somente_objeto`**: `True` se contém apenas objeto, sem background.
- **`background_e_objeto`**: `True` se a imagem contém ambas as classes.
- **`faixa_proporcao_objeto`**: Faixa da proporção de objeto (ex: `"0%"`, `"<1%"`, `"1–10%"`, etc.).
